<a href="https://colab.research.google.com/github/huzaifa678/AI-chatbot-model/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI ChatBot model

## This model was made through downloading and extracting two text datasets which includes the movie lines text informations and movie conversation text information

 ### Following steps were taken to train the model


        1.   Preprocessing:  

          * This includes converting capital letters to small letters and laters further removing the text data for the line.txt file which was not neccessary for the model to train such as "+++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++" further mapping the line ids from the conversation.txt file to their corresponding sentences from the line.txt file.

          * Further the tokenization step took place where the sentecnes were further tokenized to words and then padding with an appropriate padding size was applied to equalize the vectors for handling the data imbalance problem improving the scope of the data for the model that is to be trained.



      2.   model training and
           performance

              * The first layer for the model includes the embedding layer where the token index size + 1 was given due to indexing approach which starts at 0 and the final preprocessed data strucutre was used for the input.

              * The second layer includes the LSTM which has 512 nodes. For this layer the LSTM architecture was used to train on the sequential and contexual data which includes multiple layers of words context required long term memory along with short term memory as it was the most suitable model for this dataset.

              * The third layer includes the deep dense layer which includes 512 nodes and the relu activation function.
              
              * The last layer includes the same deep dense layer where the softmax activation function was used as the final output.



      3.   model performace

            The model performed with an overall 10% accuracy.


# Further note: This model will be improved with further updates. This model is NOT finalized.





















In [7]:
!pip install tensorflow keras numpy pandas nltk


In [8]:
def load_movie_lines(file_path):
    with open(file_path, encoding='iso-8859-1') as f:
        lines = f.readlines()
    return lines

def load_conversations(file_path):
    with open(file_path, encoding='iso-8859-1') as f:
        conversations = f.readlines()
    return conversations
movie_lines = load_movie_lines("/movie_lines.txt")
conversation_lines = load_conversations("/movie_conversations.txt")


# New Section

In [9]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')

lines = open('/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conversations = open('/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
conversations[1]

"u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']"

In [11]:
lines[1]

'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!'

In [12]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def load_movie_lines(file_path):
    with open(file_path, encoding='iso-8859-1') as f:
        lines = f.readlines()
    return lines

def load_conversations(file_path):
    with open(file_path, encoding='iso-8859-1') as f:
        conversations = f.readlines()
    return conversations

def extract_conversation_ids(conversations):
    conversation_ids = []
    for conversation in conversations[:-1]:
        _conversation = conversation.split(' +++$+++ ')[-1]
        _conversation = _conversation.strip()[1:-1].replace("'", "").replace(" ", "")
        conversation_ids.append(_conversation.split(','))
    return conversation_ids


def create_pairs(conversation_ids, lines_dict):
    questions, answers = [], []
    for conversation in conversation_ids:
        for i in range(len(conversation) - 1):
            questions.append(lines_dict[conversation[i]])
            answers.append(lines_dict[conversation[i + 1]])
    return questions, answers

lines_dict = {}
for line in lines:
    parts = line.split(" +++$+++ ")
    if len(parts) == 5:
        line_id, dialogue = parts[0], parts[4]
        lines_dict[line_id] = clean_text(dialogue)

conversation_ids = extract_conversation_ids(conversations)

print(conversation_ids[0])

questions, answers = create_pairs(conversation_ids, lines_dict)

print("Sample question:", questions[0])
print("Sample answer:", answers[0])

['L194', 'L195', 'L196', 'L197']
Sample question: can we make this quick roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad again
Sample answer: well i thought wed start with pronunciation if thats okay with you


In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_data(questions, answers, max_sequence_len):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(questions + answers)

    question_seq = tokenizer.texts_to_sequences(questions)
    answer_seq = tokenizer.texts_to_sequences(answers)

    question_seq = pad_sequences(question_seq, maxlen=max_sequence_len, padding='post')
    answer_seq = pad_sequences(answer_seq, maxlen=max_sequence_len, padding='post')

    return question_seq, answer_seq, tokenizer

max_sequence_len = 25
question_seq, answer_seq, tokenizer = preprocess_data(questions, answers, max_sequence_len)

print("Sample tokenized question:", question_seq[0])
print("Sample tokenized answer:", answer_seq[0])
print("Word index:", tokenizer.word_index)


Sample tokenized question: [   51    26   108    15   977 46849 46850     7  4068  8375    33   411
    75  3859 22221   965   510    47    29     3 30845   190     0     0
     0]
Sample tokenized answer: [   46     2   142   632   320    32 30846    41    50   112    32     1
     0     0     0     0     0     0     0     0     0     0     0     0
     0]
Word index: {'you': 1, 'i': 2, 'the': 3, 'to': 4, 'a': 5, 'it': 6, 'and': 7, 'of': 8, 'that': 9, 'in': 10, 'what': 11, 'me': 12, 'is': 13, 'dont': 14, 'this': 15, 'for': 16, 'do': 17, 'im': 18, 'know': 19, 'have': 20, 'your': 21, 'my': 22, 'not': 23, 'was': 24, 'no': 25, 'we': 26, 'he': 27, 'be': 28, 'on': 29, 'its': 30, 'but': 31, 'with': 32, 'are': 33, 'just': 34, 'all': 35, 'like': 36, 'about': 37, 'get': 38, 'youre': 39, 'so': 40, 'if': 41, 'out': 42, 'they': 43, 'here': 44, 'him': 45, 'well': 46, 'up': 47, 'got': 48, 'want': 49, 'thats': 50, 'can': 51, 'think': 52, 'were': 53, 'at': 54, 'how': 55, 'there': 56, 'one': 57, 'right

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128))
model.add(LSTM(200, return_sequences=False))
model.add(LSTM(200, return_sequences=False))
model.add(Dense(512, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

all_texts = questions + answers

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)

question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

max_sequence_len = 25
X = pad_sequences(question_sequences, maxlen=max_sequence_len, padding='post')

y = np.array([seq[0] for seq in answer_sequences if len(seq) > 0])

if len(y) != len(X):
    min_length = min(len(X), len(y))
    X = X[:min_length]
    y = y[:min_length]

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

vocab_size = len(tokenizer.word_index) + 1
print(f"Vocabulary size: {vocab_size}")

Shape of X: (221411, 25)
Shape of y: (221411,)
Vocabulary size: 65607


In [ ]:
!pip install keras-tuner


In [ ]:
history = model.fit(X, y, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
2768/2768 ━━━━━━━━━━━━━━━━━━━━ 198s 69ms/step - accuracy: 0.0970 - loss: 5.8761 - val_accuracy: 0.0976 - val_loss: 5.6826
Epoch 2/10
2768/2768 ━━━━━━━━━━━━━━━━━━━━ 200s 68ms/step - accuracy: 0.0987 - loss: 5.5072 - val_accuracy: 0.0976 - val_loss: 5.7549
Epoch 3/10
2768/2768 ━━━━━━━━━━━━━━━━━━━━ 203s 69ms/step - accuracy: 0.0973 - loss: 5.4833 - val_accuracy: 0.0976 - val_loss: 5.7981
Epoch 4/10
2768/2768 ━━━━━━━━━━━━━━━━━━━━ 201s 68ms/step - accuracy: 0.0975 - loss: 5.4447 - val_accuracy: 0.0976 - val_loss: 5.8531
Epoch 5/10
2768/2768 ━━━━━━━━━━━━━━━━━━━━ 202s 68ms/step - accuracy: 0.0987 - loss: 5.4254 - val_accuracy: 0.0976 - val_loss: 5.8749
Epoch 6/10
1999/2768 ━━━━━━━━━━━━━━━━━━━━ 47s 61ms/step - accuracy: 0.1004 - loss: 5.3948

KeyboardInterrupt: 

In [ ]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
print(X.shape)

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        model = Sequential()
        model.add(Embedding(
            input_dim=vocab_size,
            output_dim=hp.Int('embedding_dim', min_value=64, max_value=256, step=64),
        ))
        model.add(LSTM(
            units=hp.Int('lstm_units', min_value=128, max_value=512, step=128)
        ))
        model.add(Dense(
            units=hp.Int('dense_units', min_value=128, max_value=512, step=128),
            activation='relu'
        ))
        model.add(Dense(vocab_size, activation='softmax'))

        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=10,
    directory='hyperparam_tuning',
    project_name='tuning_lstm_model'
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

tuner.search(X, y, epochs=10, validation_split=0.2, batch_size=64, callbacks=[early_stopping])

(221411, 25)

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
128               |128               |embedding_dim
128               |128               |lstm_units
512               |512               |dense_units

Epoch 1/10
2768/2768 ━━━━━━━━━━━━━━━━━━━━ 188s 67ms/step - accuracy: 0.0971 - loss: 5.9160 - val_accuracy: 0.0976 - val_loss: 5.6909
Epoch 2/10
2768/2768 ━━━━━━━━━━━━━━━━━━━━ 199s 66ms/step - accuracy: 0.0994 - loss: 5.4918 - val_accuracy: 0.0976 - val_loss: 5.7353
Epoch 3/10
2768/2768 ━━━━━━━━━━━━━━━━━━━━ 177s 64ms/step - accuracy: 0.0998 - loss: 5.4493 - val_accuracy: 0.0976 - val_loss: 5.7875
Epoch 4/10
2768/2768 ━━━━━━━━━━━━━━━━━━━━ 202s 64ms/step - accuracy: 0.0985 - loss: 5.4296 - val_accuracy: 0.0976 - val_loss: 5.8599
Epoch 1/10
1013/2768 ━━━━━━━━━━━━━━━━━━━━ 1:42 58ms/step - accuracy: 0.0994 - loss: 6.2493

KeyboardInterrupt: 